# Diving Deeper into Weights & Biases

<!--- @wandbcode{mlops-zoomcamp} -->

In this notebook, we will explore the following

* Versioning datasets using [Artifacts](https://docs.wandb.ai/guides/artifacts).
* Exploring and visualizing our datasets with [Tables](https://docs.wandb.ai/guides/data-vis).
* Baseline Experiment with a Random Forest Classification Model.

## Import the Libraries

In [ ]:
import os
import pickle

import wandb
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

## Logging Dataset to Artifacts

Download the `train.csv` and `test.csv` files from [Titanic - Machine Learning from Disaster](https://www.kaggle.com/competitions/titanic/data) and place them in the `data` directory.

In [ ]:
# Initialize a WandB Run
wandb.init(project="mlops-zoomcamp-wandb", job_type="log_data")

# Log the `data` directory as an artifact
artifact = wandb.Artifact('Titanic', type='dataset', metadata={"Source": "https://www.kaggle.com/competitions/titanic/data"})
artifact.add_dir('data')
wandb.log_artifact(artifact)

# End the WandB Run
wandb.finish()

## Versioning the Data

In [ ]:
# Initialize a WandB Run
wandb.init(project="mlops-zoomcamp-wandb", job_type="log_data")

# Fetch the dataset artifact 
artifact = wandb.use_artifact('geekyrakshit/mlops-zoomcamp-wandb/Titanic:v0', type='dataset')
artifact_dir = artifact.download()

Read the dataset files

In [ ]:
train_df = pd.read_csv(os.path.join(artifact_dir, "train.csv"))
test_df = pd.read_csv(os.path.join(artifact_dir, "test.csv"))

In [ ]:
num_train_examples = int(0.8 * len(train_df))
num_val_examples = len(train_df) - num_train_examples

print(num_train_examples, num_val_examples)

In [ ]:
train_df["Split"] = ["Train"] * num_train_examples + ["Validation"] * num_val_examples
train_df.to_csv("data/train.csv", encoding='utf-8', index=False)

In [ ]:
# Log the `data` directory as an artifact
artifact = wandb.Artifact('Titanic', type='dataset', metadata={"Source": "https://www.kaggle.com/competitions/titanic/data"})
artifact.add_dir('data')
wandb.log_artifact(artifact)

# End the WandB Run
wandb.finish()

## Explore the Dataset

In [ ]:
# Initialize a WandB Run
wandb.init(project="mlops-zoomcamp-wandb", job_type="explore_data")

# Fetch the latest version of the dataset artifact 
artifact = wandb.use_artifact('geekyrakshit/mlops-zoomcamp-wandb/Titanic:latest', type='dataset')
artifact_dir = artifact.download()

# Read the files
train_val_df = pd.read_csv(os.path.join(artifact_dir, "train.csv"))
test_df = pd.read_csv(os.path.join(artifact_dir, "test.csv"))

In [ ]:
# Create tables corresponding to datasets
train_val_table = wandb.Table(dataframe=train_val_df)
test_table = wandb.Table(dataframe=test_df)

# Log the tables to Weights & Biases
wandb.log({
    "Train-Val-Table": train_val_table,
    "Test-Table": test_table
})

# End the WandB Run
wandb.finish()

## Fit a Baseline Model

In [ ]:
# Initialize a WandB Run
wandb.init(project="mlops-zoomcamp-wandb", name="baseline_experiment-2", job_type="train")

# Fetch the latest version of the dataset artifact 
artifact = wandb.use_artifact('geekyrakshit/mlops-zoomcamp-wandb/Titanic:latest', type='dataset')
artifact_dir = artifact.download()

# Read the files
train_val_df = pd.read_csv(os.path.join(artifact_dir, "train.csv"))
test_df = pd.read_csv(os.path.join(artifact_dir, "test.csv"))

In [ ]:
features = ["Pclass", "Sex", "SibSp", "Parch"]
X_train = pd.get_dummies(train_val_df[features][train_val_df["Split"] == "Train"])
X_val = pd.get_dummies(train_val_df[features][train_val_df["Split"] == "Validation"])
y_train = train_val_df["Survived"][train_val_df["Split"] == "Train"]
y_val = train_val_df["Survived"][train_val_df["Split"] == "Validation"]

In [ ]:
model_params = {"n_estimators": 100, "max_depth": 10, "random_state": 1}
wandb.config = model_params

model = RandomForestClassifier(**model_params)
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_probas_train = model.predict_proba(X_train)
y_pred_val = model.predict(X_val)
y_probas_val = model.predict_proba(X_val)

In [ ]:
wandb.log({
    "Train/Accuracy": accuracy_score(y_train, y_pred_train),
    "Validation/Accuracy": accuracy_score(y_val, y_pred_val),
    "Train/Presicion": precision_score(y_train, y_pred_train),
    "Validation/Presicion": precision_score(y_val, y_pred_val),
    "Train/Recall": recall_score(y_train, y_pred_train),
    "Validation/Recall": recall_score(y_val, y_pred_val),
    "Train/F1-Score": f1_score(y_train, y_pred_train),
    "Validation/F1-Score": f1_score(y_val, y_pred_val),
})

In [ ]:
label_names = ["Not-Survived", "Survived"]

wandb.sklearn.plot_class_proportions(y_train, y_val, label_names)
wandb.sklearn.plot_summary_metrics(model, X_train, y_train, X_val, y_val)
wandb.sklearn.plot_roc(y_val, y_probas_val, labels=label_names)
wandb.sklearn.plot_precision_recall(y_val, y_probas_val, labels=label_names)
wandb.sklearn.plot_confusion_matrix(y_val, y_pred_val, labels=label_names)

In [ ]:
# Save your model
with open("random_forest_classifier.pkl", "wb") as f:
    pickle.dump(model, f)

# Log your model as a versioned file to Weights & Biases Artifact
artifact = wandb.Artifact(f"titanic-random-forest-model", type="model")
artifact.add_file("random_forest_classifier.pkl")
wandb.log_artifact(artifact)


# End the WandB Run
wandb.finish()